<a href="https://colab.research.google.com/github/connerbjoyce/hacker_network_scripts/blob/main/BERT_OPPCNTRY_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Jun 30 21:07:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# install packages
!pip install --upgrade transformers
!pip install pyreadr
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 47.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
t

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from datasets import Dataset
import torch
import transformers as ppb
import warnings
import pyreadr
from transformers import DistilBertTokenizer, TextClassificationPipeline, AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, BertTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [ ]:
# mount google colab to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load test and train dataset
test_set = pyreadr.read_r('/content/drive/My Drive/BERT TEST Folder/test_set.RData')
first_key = next(iter(test_set))
test_set = test_set[first_key]
test_set.head()

train_set = pyreadr.read_r('/content/drive/My Drive/BERT_OPPCNTRY Folder/train_OPPCNTRY.RData')
first_key = next(iter(train_set))
train_set = train_set[first_key]
train_set.head()


,content_id_train,text_translate_only,OPP_CNTRY_BIN
0,1,,0.0
1,2,,0.0
2,3,,0.0
3,4,...,0.0
4,5,...,0.0


In [ ]:
train_set = train_set.rename(columns={'OPP_CNTRY_BIN': 'label'})

In [ ]:
train_set['label'] = train_set['label'].astype(int)
train_set.head()

,content_id_train,text_translate_only,label
0,1,,0
1,2,,0
2,3,,0
3,4,...,0
4,5,...,0


In [ ]:
# split train and val set
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=42)
len(train_set)
len(val_set)

425

In [ ]:
id2label = {0: "NONE", 1: "OPPOSE"}
label2id = {"NONE": 0, "OPPOSE": 1}

In [ ]:
# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', max_length=512, truncation=True)
#tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2,
                                                      id2label=id2label, label2id=label2id)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_set = Dataset.from_pandas(train_set)
test_set = Dataset.from_pandas(test_set)
val_set = Dataset.from_pandas(val_set)

def preprocess_function(examples):
    return tokenizer(examples["text_translate_only"], truncation=True, padding='max_length')

tokenized_train = train_set.map(preprocess_function, batched=True)
tokenized_test = test_set.map(preprocess_function, batched=True)
tokenized_val = val_set.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#train_data = train_set.map(lambda e: tokenizer(e['text_translate_only'], truncation=True, padding='max_length'), batched=True)
#test_data = test_set.map(lambda e: tokenizer(e['text_translate_only'], truncation=True, padding='max_length'), batched=True)


Map:   0%|          | 0/1700 [00:00<?, ? examples/s]

Map:   0%|          | 0/34065 [00:00<?, ? examples/s]

Map:   0%|          | 0/425 [00:00<?, ? examples/s]

In [ ]:
#train_data.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
#train_data.format['type']
#test_data.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
#test_data.format['type']

In [ ]:
# preprocess steps
#tokenized_train = train_set['text_translate_only'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#tokenized_test = test_set['text_translate_only'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Training Args
training_args = TrainingArguments(
    #output_dir='./results',          # output directory for model predictions and checkpoints
    #evaluation_strategy="epoch",     # evaluate at the end of each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=2,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    report_to="none"
)

In [ ]:
# Train model (
trainer = Trainer(
    model=model,                        # the model to be trained
    args=training_args,                 # training arguments
    train_dataset=tokenized_train,
    processing_class=tokenizer,
    data_collator=data_collator,        # training dataset
    eval_dataset=tokenized_val,             # evaluation dataset)
    compute_metrics=compute_metrics
    )

In [ ]:
# (do not need to run when reloading)
trainer.train()

Step,Training Loss
10,0.670300
20,0.338400
30,0.133600
40,0.202200
50,0.518300
60,0.264900
70,0.499000
80,0.279800
90,0.721800
100,0.438900


TrainOutput(global_step=2550, training_loss=0.1275660930265782, metrics={'train_runtime': 651.4446, 'train_samples_per_second': 7.829, 'train_steps_per_second': 3.914, 'total_flos': 1341866382336000.0, 'train_loss': 0.1275660930265782, 'epoch': 3.0})

In [ ]:
# Create a folder in the root directory
#!mkdir -p "/content/drive/My Drive/BERT_SUPCNTRY Folder"

In [ ]:
# check that doc is saved properly to this directory
#with open('/content/drive/My Drive/My Folder/sample.txt', 'w') as f:
  #f.write('Hello, World!')

In [ ]:
# save fine-tuned model
trainer.save_model("/content/drive/My Drive/BERT_OPPCNTRY Folder")

In [ ]:
# load fine tuned model
#trainer = Trainer(model=model,
                  #args=training_args)

In [ ]:
trainer.evaluate()

{'eval_loss': 0.050100523978471756,
 'eval_accuracy': 0.9929411764705882,
 'eval_precision': 1.0,
 'eval_recall': 0.7857142857142857,
 'eval_f1': 0.88,
 'eval_runtime': 13.3459,
 'eval_samples_per_second': 31.845,
 'eval_steps_per_second': 15.96,
 'epoch': 3.0}

In [ ]:
# create predictions from fine tuned model
pipe = TextClassificationPipeline(model=model, tokenizer = tokenizer, truncation=True, max_length=512)

# Remove 'token_type_ids' from the inputs to the model

prediction = pipe(test_set['text_translate_only'], return_all_scores=True)
#y = trainer.predict(tokenized_test)

Device set to use cuda:0


In [ ]:
#print(prediction)

In [ ]:
# Flatten the list (extract dictionaries from the nested list)
flat_data = [item for sublist in prediction for item in sublist]

# Create a dictionary for each row with 'NONE' and 'SUPPORT' as keys and their scores as values
rows = {
    'NONE_score': [],
    'OPPOSE_score': []
}

for i in range(0, len(flat_data), 2):  # Step by 2 to pair NONE and SUPPORT
    none_score = next(item['score'] for item in flat_data[i:i+2] if item['label'] == 'NONE')
    support_score = next(item['score'] for item in flat_data[i:i+2] if item['label'] == 'OPPOSE')

    rows['NONE_score'].append(none_score)
    rows['OPPOSE_score'].append(support_score)

# Convert to DataFrame
df = pd.DataFrame(rows)

# reload test
test_set = pyreadr.read_r('/content/drive/My Drive/BERT TEST Folder/test_set.RData')
first_key = next(iter(test_set))
test_set = test_set[first_key]

# concat with test data
df_combined = pd.concat([test_set, df], axis=1)

# Display the DataFrame
print(df_combined)

                                     text_translate_only  content_id_test  \
0      hacked by temp    anonsec team anon    lananak...                1   
1      hacked by rikx    we smile in your system     ...                2   
2      hacked by anonpeople  oops  owned by semarang ...                3   
3      hacked by tn lord  greets   uchihadarknet lamp...                4   
4      hacked by wans  crazy money   for someone   i ...                5   
...                                                  ...              ...   
34060  hacked by   mr spongebob security   it s just ...            36076   
34061  hacked by xyzeaz   xyz                        ...            36077   
34062  linux can t sell     amd    smp preempt dynami...            36078   
34063  hacked by tcodex   contact   info psb close me...            36079   
34064  hello administrator   hacked by   pian xd   oo...            36080   

       NONE_score  OPPOSE_score  
0        0.999904      0.000096  
1      

In [ ]:
df_combined.to_csv('/content/drive/My Drive/BERT_OPPCNTRY Folder/labels_OPPCNTRY_base.csv', index=False)
